In [1]:
import librosa
import librosa.display
import json
import numpy as np
import pandas as pd
import os
import pickle

In [2]:
audio = "F:/ravdess_tess/"
actor_folders = os.listdir(audio) #list files in audio directory
actor_folders.sort() 
actor_folders[:5]

['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05']

In [3]:
emotion = []
gender = []
actor = []
file_path = []
for i in actor_folders:
    filename = os.listdir(audio + i) #in actor folder
    for f in filename: #in files of actor folder
        part = f.split('.')[0].split('-')
        emotion.append(int(part[2]))
        actor.append(int(part[6]))
        bg = int(part[6])
        if bg%2 == 0:
            bg = "female"
        else:
            bg = "male"
        gender.append(bg)
        file_path.append(audio + i + '/' + f)

In [12]:
audio_df = pd.DataFrame(emotion)
#audio_df = audio_df.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
audio_df = pd.concat([pd.DataFrame(gender),audio_df,pd.DataFrame(actor)],axis=1)
audio_df.columns = ['gender','emotion','actor']
audio_df = pd.concat([audio_df,pd.DataFrame(file_path, columns = ['path'])],axis=1)
audio_df

,gender,emotion,actor,path
0,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-01-01-01.wav
1,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-01-02-01.wav
2,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-02-01-01.wav
3,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-02-02-01.wav
4,male,calm,1,F:/ravdess_tess/Actor_01/03-01-02-01-01-01-01.wav
...,...,...,...,...
5247,male,sad,71,F:/ravdess_tess/Actor_26/98-57-04-35-61-19-71.wav
5248,female,sad,18,F:/ravdess_tess/Actor_26/98-60-04-95-21-75-18.wav
5249,female,fear,26,F:/ravdess_tess/Actor_26/98-65-06-32-90-87-26.wav
5250,male,neutral,23,F:/ravdess_tess/Actor_26/98-71-01-30-75-41-23.wav


In [14]:
df = pd.DataFrame(columns=['feature'])

# loop feature extraction over the entire dataset
counter=0
for index,path in enumerate(audio_df.path):
    X, sample_rate = librosa.load(path, res_type='kaiser_fast')
    sample_rate = np.array(sample_rate)
 
    mfccs = np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=40).T,axis=0)
    df.loc[counter] = [mfccs]
    counter=counter+1   
    

# Checkthe first few recordings
print(len(df))
df.head()

5252


,feature
0,"[-700.3989, 58.63021, -3.025852, 16.040241, 4...."
1,"[-695.55786, 59.240154, -5.3727765, 19.776367,..."
2,"[-694.00433, 61.49651, -3.2627435, 16.971298, ..."
3,"[-687.51337, 59.44154, -0.703714, 16.645708, 3..."
4,"[-729.98016, 66.51589, -0.9419841, 19.070974, ..."


In [15]:
df = pd.concat([audio_df,pd.DataFrame(df['feature'].values.tolist())],axis=1)
df[:5]

,gender,emotion,actor,path,0,1,2,3,4,5,...,30,31,32,33,34,35,36,37,38,39
0,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-01-01-01.wav,-700.398926,58.630211,-3.025852,16.040241,4.248529,3.869935,...,-1.411359,-2.769772,-2.042009,-2.522663,-2.507448,-2.250499,-0.381507,-2.481059,-2.791023,-2.244865
1,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-01-02-01.wav,-695.557861,59.240154,-5.372777,19.776367,5.200387,3.324630,...,-1.055913,-3.102514,-1.697880,-2.922661,-2.544465,-1.289832,-0.797254,-3.586074,-2.706395,-2.812933
2,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-02-01-01.wav,-694.004333,61.496510,-3.262743,16.971298,2.142968,4.266798,...,-1.382858,-2.590943,-1.929074,-2.390322,-2.269381,-2.486079,-0.589257,-3.248326,-2.979813,-2.769281
3,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-02-02-01.wav,-687.513367,59.441540,-0.703714,16.645708,3.730826,6.181194,...,-2.024173,-2.586190,-2.420421,-3.243219,-2.762588,-1.960003,-0.453890,-2.976706,-2.914763,-3.909605
4,male,calm,1,F:/ravdess_tess/Actor_01/03-01-02-01-01-01-01.wav,-729.980164,66.515892,-0.941984,19.070974,4.297080,5.635082,...,-1.519869,-1.797158,-1.296814,-3.051891,-1.131349,-1.063672,-1.141021,-2.373389,-3.204345,-3.363193


In [16]:
df=df.fillna(0)
df.to_csv(r'./mfcc_data.csv',index=False, header=True)
print(df.shape)
df[:5]

(5252, 44)


,gender,emotion,actor,path,0,1,2,3,4,5,...,30,31,32,33,34,35,36,37,38,39
0,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-01-01-01.wav,-700.398926,58.630211,-3.025852,16.040241,4.248529,3.869935,...,-1.411359,-2.769772,-2.042009,-2.522663,-2.507448,-2.250499,-0.381507,-2.481059,-2.791023,-2.244865
1,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-01-02-01.wav,-695.557861,59.240154,-5.372777,19.776367,5.200387,3.324630,...,-1.055913,-3.102514,-1.697880,-2.922661,-2.544465,-1.289832,-0.797254,-3.586074,-2.706395,-2.812933
2,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-02-01-01.wav,-694.004333,61.496510,-3.262743,16.971298,2.142968,4.266798,...,-1.382858,-2.590943,-1.929074,-2.390322,-2.269381,-2.486079,-0.589257,-3.248326,-2.979813,-2.769281
3,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-02-02-01.wav,-687.513367,59.441540,-0.703714,16.645708,3.730826,6.181194,...,-2.024173,-2.586190,-2.420421,-3.243219,-2.762588,-1.960003,-0.453890,-2.976706,-2.914763,-3.909605
4,male,calm,1,F:/ravdess_tess/Actor_01/03-01-02-01-01-01-01.wav,-729.980164,66.515892,-0.941984,19.070974,4.297080,5.635082,...,-1.519869,-1.797158,-1.296814,-3.051891,-1.131349,-1.063672,-1.141021,-2.373389,-3.204345,-3.363193


In [17]:
dataframe=pd.read_csv('./mfcc_data.csv')
print(dataframe.shape)
dataframe.head()

(5252, 44)


,gender,emotion,actor,path,0,1,2,3,4,5,...,30,31,32,33,34,35,36,37,38,39
0,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-01-01-01.wav,-700.398926,58.630211,-3.025852,16.040241,4.248529,3.869935,...,-1.411359,-2.769772,-2.042009,-2.522663,-2.507448,-2.250499,-0.381507,-2.481059,-2.791023,-2.244865
1,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-01-02-01.wav,-695.557861,59.240154,-5.372777,19.776367,5.200387,3.324630,...,-1.055913,-3.102514,-1.697880,-2.922661,-2.544465,-1.289832,-0.797254,-3.586074,-2.706395,-2.812933
2,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-02-01-01.wav,-694.004333,61.496510,-3.262743,16.971298,2.142968,4.266798,...,-1.382858,-2.590943,-1.929074,-2.390322,-2.269381,-2.486079,-0.589257,-3.248326,-2.979813,-2.769281
3,male,neutral,1,F:/ravdess_tess/Actor_01/03-01-01-01-02-02-01.wav,-687.513367,59.441540,-0.703714,16.645708,3.730826,6.181194,...,-2.024173,-2.586190,-2.420421,-3.243219,-2.762588,-1.960003,-0.453890,-2.976706,-2.914763,-3.909605
4,male,calm,1,F:/ravdess_tess/Actor_01/03-01-02-01-01-01-01.wav,-729.980164,66.515892,-0.941984,19.070974,4.297080,5.635082,...,-1.519869,-1.797158,-1.296814,-3.051891,-1.131349,-1.063672,-1.141021,-2.373389,-3.204345,-3.363193
